# Runing VQE on Braket backend

In [1]:
from qiskit.algorithms import VQE
from qiskit.opflow import (
    I,
    X,
    Z,
)
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance, algorithm_globals

from qiskit_braket_plugin import AWSBraketProvider, BraketLocalBackend

seed = 50
algorithm_globals.random_seed = seed

Get backend to run VQE with

In [2]:
provider = AWSBraketProvider()
local_simulator = BraketLocalBackend()
local_simulator

BraketBackend[sv_simulator]

In [4]:
state_vector_simulator_backend = provider.get_backend("SV1")
state_vector_simulator_backend

BraketBackend[SV1]

Running VQE



More docs on VQE and algorithms https://qiskit.org/documentation/tutorials/algorithms/01_algorithms_introduction.html#A-complete-working-example

In [5]:
H2_op = (
    (-1.052373245772859 * I ^ I)
    + (0.39793742484318045 * I ^ Z)
    + (-0.39793742484318045 * Z ^ I)
    + (-0.01128010425623538 * Z ^ Z)
    + (0.18093119978423156 * X ^ X)
)

qi = QuantumInstance(
    state_vector_simulator_backend, seed_transpiler=seed, seed_simulator=seed
)
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
slsqp = SLSQP(maxiter=1)

vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi)

result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 9,
    'eigenstate': {   '00': 0.7824990015968072,
                      '01': 0.489139870078079,
                      '10': 0.3814548629916782,
                      '11': 0.05412658773652741},
    'eigenvalue': (-1.0606393547183097+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[1]): 4.19301252102391,
                              ParameterVectorElement(θ[0]): 3.611860069224077,
                              ParameterVectorElement(θ[2]): 0.6019852007557844,
                              ParameterVectorElement(θ[3]): 5.949536809130025,
                              ParameterVectorElement(θ[4]): -3.3070470445355764,
                              ParameterVectorElement(θ[6]): -5.466043598406607,
                              ParameterVectorElement(θ[7]): 0.6984088030463615,
                              ParameterVectorElement(θ[5]): 1.8462931831829383},
    'optimal_point': array([ 3.61186007,  4.19301252,  0.